Lab 2 - Week 1 | Day 3

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown,display

In [ ]:
load_dotenv(override=True)

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenRouter API Key not set")

In [ ]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

print(messages)

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model = 'gpt-4o-mini',
    messages=messages
)

question = response.choices[0].message.content
print(question)

In [ ]:
# Define a list of competitors and their answers for comaprison
competitors = []
answers = []

# Define the common messages in OpenAI format
messages = [{"role": "user", "content":question}]

In [ ]:
# OpenAI - The API we know well - 

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Leverage OpenRouter to use all other Models referenced in the lab

In [ ]:
# Create openRouter client. This follows the standard OpenAI format with a base URL to openrouter
openrouter_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key, # Replace with your actual key
)

In [ ]:
# Anthropic has a slightly different API, and requires Max Tokens as a param

# claude = Anthropic()
# response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
# answer = response.content[0].text

# But now, we are leveraging the OpenRouter API to call Anthropic API instead

claude=openrouter_client
model_name = "anthropic/claude-3-7-sonnet"

response = claude.chat.completions.create(
        model=model_name, # Example: Using Anthropic model
        messages=messages,
        # max_tokens=1000 not required when accessed in openRouter
)

answer = response.choices[0].message.content
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)

In [ ]:
# Gemini 

gemini = openrouter_client
model_name = "google/gemini-2.5-flash"

response = gemini.chat.completions.create(
        model=model_name, 
        messages=messages,
)

answer = response.choices[0].message.content
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)

In [ ]:
# Deepseek

deepseek = openrouter_client
model_name = "deepseek/deepseek-chat"

response = deepseek.chat.completions.create(
        model=model_name, 
        messages=messages,
)

answer = response.choices[0].message.content
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)


In [ ]:
# Groq - This is a provider of fast inference on Specialized hardware.
# This is a platform that supports multiple models.
# The model "llama-3.3-70b-versatile" is only available in Groq but I don't have a Groq key.
# So I am using Meta's "meta-llama/llama-3.3-70b-instruct" model in OpenRouter as an alternative competitor.

groq = openrouter_client
model_name = "meta-llama/llama-3.3-70b-instruct"

response = groq.chat.completions.create(
        model=model_name, 
        messages=messages,
)

answer = response.choices[0].message.content
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)

I am skipping Ollama local installation for now - It's cod is available in Lab 2 notebook

In [ ]:
# So where are we?

print(competitors)
print(answers)

In [ ]:
# It's nice to know how to use "zip"
# Print the competitors and their answers as a pair

for competitor, answer in zip(competitors, answers):
    message_pair=f"Competitor: {competitor}\n\n{answer}"
    display(Markdown(message_pair))
    # print(message_pair)
    # print(f"Competitor: {competitor}\n\n{answer}")

In [ ]:
# Print all the answers together in a single markdown block
together = ""
for index, answer in enumerate(answers):
    together += f"### Response from competitor {index+1}\n\n"
    together += answer

display(Markdown(together))
# print(together)

Judgement Time

In [ ]:
# Judgement instruction

judge_instruction = ""

judge_instruction+=f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. 
Do not include markdown formatting or code blocks."""

judge_messages = [{"role": "user", "content": judge_instruction}]

In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content

print(results)

In [ ]:
# Let's turn this into readable results!

results_dict = json.loads(results)
winners = results_dict["results"]

# print(winners)
# print(competitors)

for index, winner in enumerate(winners):
    # print(index, winner)
    competitor = competitors[int(winner)-1]
    print(f"Rank {index+1}: {competitor}")
